# IMT‑211 · Análisis Cinemático de Mecanismos de 4 Barras

Cuaderno base para **Mecanismos**. Permite:
- Definir un mecanismo **cuadrilátero articulado** (4‑bar) en el plano.
- Simular el movimiento en el tiempo (posición, velocidad, aceleración) a partir de una entrada angular (crank).
- **Graficar cualquier variable** relevante en función del tiempo.
- **Animar** los eslabones y **trazar** la trayectoria de un punto de interés (similar a GeoGebra).

> Dependencias recomendadas del entorno (construidas en tu *repo* y entorno virtual): `python >=3.11`, `numpy`, `matplotlib`. Opcional: `ipywidgets` (para controles interactivos), `pandas` (para exportar resultados).


## Importación de librerias

In [1]:
import math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


## 1) Definición del mecanismo y del perfil de entrada

### Eslabones:
- `L0`: **Bancada**
- `L1`: **Biela**
- `L2`: **Acoplador**
- `L3`: **Brazo balancín**

### Juntas:
- `O_1`: **L0,R,L1**
- `A`:  **L1,R,L2**
- `B`:  **L2,R,L3**
- `O_4`: **L0,R,L3**

### Punto de Interés:
- `Y`: **Extremo del brazo del balancín**




In [ ]:
# --- Longitudes de eslabones (unidades arbitrarias, p. ej., mm) ---
l0x = -32  # logitud en x de bancada
l0y = -5
l1 = 12
l2 = 30
l3 = 25

# --- Crear las variables angulares ---
th1 = sp.symbols('th_1')
th2 = sp.symbols('th_2')
th3 = sp.symbols('th_3')

# --- definir mis verctores ---
Ro4 = sp.Matrix([[l0x],
                [l0y],
                [0]])

Ra = sp.Matrix([[l1*sp.cos(th1)],
                [l1*sp.sin(th1)],
                [0]])
## Completar vectores

# --- Crear las variables de velocidades angulares ---

# --- Crear los Vectores de velocidad --- (para angulares y escalares)


# --- Crear las variables de aceleraciones angulares ---

# --- Crear los Vectores de Aceleración --- (para angulares y escalares)


Matrix([
[12*cos(th_1)],
[12*sin(th_1)],
[           0]])